In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

##NoteBook Loader Class


In [0]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader():
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

##Main For Loading Notebooks

In [0]:
def load_main(path, filename):
  #Change pwd to directory which holds the notebook
  %cd $path 
  #example of input: '/content/drive/My Drive/COMP767/Project/Test/'

  !pwd 

  #Check that notebook exists in that directory 
  path = find_notebook(filename) #Give the name of the file before .ipynb --> example: 'Test3' (if you want a file called Test3.ipynb)
  if not path: print('FILE DOES NOT EXIST, TRY AGAIN')

  #This creates an instance of the Notebook Loader class 
  loader = NotebookLoader(sys.path) 
  if not loader: print('ERROR WITH CREATING LOADER')

  #Load Module 
  mod = loader.load_module(filename) 
  print('file available: ', mod)  




In [30]:
file_to_load = 'Test3' #Notebook name before .ipynb
file_location = '/content/drive/My Drive/COMP767/Project/Test/' #Test is parent directory of notebook
#Navigate to root directory 
%cd /content/
#Load notebook 
load_main(file_location, file_to_load)

#Import notebook 
from Test3 import * 

#Run function from notebook 
testing_notebook_loader() 


/content
/content/drive/My Drive/COMP767/Project/Test
/content/drive/My Drive/COMP767/Project/Test
importing Jupyter notebook from Test3.ipynb
file available:  <module 'Test3' from 'Test3.ipynb'>


'Hello, this is a message from Test3. You have successfully loaded the notebook!'